In [2]:
import pickle

fname = '../../data/fmri-FC-slim.pkl'
fmriDict = None

with open(fname, 'rb') as f:
    fmriDict = pickle.load(f)
    
print(fmriDict.keys())

dict_keys(['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap'])


In [3]:
def getGroupIdcs(groupsMap):
    normals = []
    fibros = []
    for num,diag in groupsMap.items():
        if diag == 1:
            normals.append(num)
        else:
            fibros.append(num)
    return normals,fibros

normals, fibros = getGroupIdcs(fmriDict['groupsNormalDiagMap'])

print(len(normals))
print(normals)
print(len(fibros))
print(fibros)

33
['007', '012', '014', '016', '018', '021', '022', '026', '030', '031', '032', '033', '034', '036', '042', '045', '047', '056', '058', '059', '060', '061', '064', '066', '068', '069', '070', '072', '073', '074', '075', '076', '077']
33
['002', '004', '005', '006', '008', '009', '010', '011', '013', '015', '017', '019', '020', '023', '024', '025', '028', '029', '037', '038', '039', '040', '043', '044', '046', '049', '050', '052', '053', '054', '055', '062', '063']


In [20]:
import numpy as np

FCslim = fmriDict['FC-slim']
subjNum2Idx = fmriDict['subjNum2IdxMap']

Xfibros = [FCslim[subjNum2Idx[num]] for num in fibros]
Xnormals = [FCslim[subjNum2Idx[num]] for num in normals]

X = np.concatenate([Xfibros, Xnormals], axis=0)
X = X.T

y = np.concatenate([np.ones((1,33)), -np.ones((1,33))], axis=1)
y = np.repeat(y, 34716, axis=0)

print(y.shape)
print(X.shape)

(34716, 66)
(34716, 66)


In [21]:
sxy = np.einsum('ab,ab->a',X,y)
sxx = np.einsum('ab,ab->a',X,X)
syy = np.einsum('ab,ab->a',y,y)

rho = sxy/np.sqrt(sxx*syy)

print(rho.shape)

(34716,)


In [22]:
idcs = np.argsort(np.abs(rho))
print(idcs[-10:])
print(rho[idcs[-10:]])

[23762 16198  2412  2437 22309 12379  2426 22846  2352  2436]
[-0.41117489 -0.41245927 -0.42228221 -0.42969494 -0.43587849 -0.43924892
 -0.4582674  -0.45887354 -0.46873259 -0.52940131]


In [23]:
a,b = np.triu_indices(264,1)
print(a[idcs[-20:]])
print(b[idcs[-20:]])
lst = []
for idx in idcs[-20:]:
    lst.append(f'{a[idx]}-{b[idx]}')
for conn in lst[::-1]:
    print(conn)

[184 160 136   1 101 115 123  25  26   9 115  71   9   9 105  52   9 109
   9   9]
[195 219 185  91 116 189 219 222 115 108 188  82  91 116 260  82 105 175
  31 115]
9-115
9-31
109-175
9-105
52-82
105-260
9-116
9-91
71-82
115-188
9-108
26-115
25-222
123-219
115-189
101-116
1-91
136-185
160-219
184-195
